In [ ]:
!git clone https://github.com/you-ming-hu/Cardiac_Catheterization_Project
!pip install git+https://github.com/qubvel/segmentation_models.pytorch
!pip install torch==1.10.2
!pip install torchinfo
import os
os.chdir('./Cardiac_Catheterization_Project/pytorch')
from IPython.display import clear_output
clear_output()
!nvidia-smi
!nvcc --version

In [ ]:
import Schedule.Transform

from utils.configuration import Editor

with Editor('Config') as Config:

    # settings for dataset
    Config.Dataset.ImagesRootPath = '/kaggle/input/cardiac-catheterization/Cardiac_Catheterization/train/images'
    Config.Dataset.MasksRootPath = '/kaggle/input/cardiac-catheterization/Cardiac_Catheterization/train/masks'
    # control input and output image format
    Config.Dataset.IO.InputRGBImage = False
    Config.Dataset.IO.NumWorkers = 0
    Config.Dataset.IO.PinMemory = False
    Config.Dataset.IO.PrefetchFactor = 2
    Config.Dataset.IO.OutputDtype = 'float'
    # uniformed preprocess
    Config.Dataset.Preprocess.Version = 'v1'
    #train dataset
    Config.Dataset.Train.BatchSize = 8
    Config.Dataset.Train.Transform.Combination.Version = 'v1'
    Config.Dataset.Train.Transform.Combination.Components = 'default'
    Config.Dataset.Train.Transform.Combination.Params = 'default'
    Config.Dataset.Train.Transform.Combination.Schedules = 'default'
    Config.Dataset.Train.Transform.Schedule = 0.8
    #validation dataset
    Config.Dataset.Validation.BatchSize = 8
    Config.Dataset.Validation.Ratio = 0.05
    Config.Dataset.Validation.Transform.Combination.Version = 'v1'
    Config.Dataset.Validation.Transform.Combination.Components = 'default'
    Config.Dataset.Validation.Transform.Combination.Params = 'default'
    Config.Dataset.Validation.Transform.Combination.Schedules = 'default'
    Config.Dataset.Validation.Transform.Schedule = 0.8
    
    #choose training structure, including the model, loss, metrics, optimizer, schedular
    Config.Training.Task = 'SimpleSeg'
    # model
    Config.Training.Model.Backbone.Name = 'Unet'
    Config.Training.Model.Backbone.Param = dict(encoder_name='densenet201', encoder_weights=None, in_channels=1, out_channels=4)
    Config.Training.Model.Head.Name = 'Head1'
    Config.Training.Model.Head.Param = dict(logit_output=True, in_channels=4)
    # loss
    Config.Training.Loss.Name = 'DiceBCELoss'
    Config.Training.Loss.Param = dict(use_logit=True,w_bce=0.2)
    # optimizer
    Config.Training.Optimizer.Name = 'Adam'
    Config.Training.Optimizer.Param = dict(lr=0.001)
    # scheduler
    Config.Training.Scheduler.Name = 'Custom_1'
    Config.Training.Scheduler.Param = dict(warmup_epochs=1,reduce_gamma=-2)
    # metrics
    Config.Training.Metrics.Name = ['DiceBCELoss','DiceAccuracy']
    Config.Training.Metrics.Param = [dict(use_logit=True,w_bce=0.2),dict(use_logit=True,threshold=0.5)]
    
    # whether to freeze backbone
    Config.Training.Settings.Model.FreezeBackbone = False
    
    # setup training process
    Config.Training.Settings.Epochs = 30
    Config.Training.Settings.GradientAccumulation = 1
    Config.Training.Settings.AmpScaleTrain = True
    # set random property
    Config.Training.Settings.Random.cuDNN.Deterministic = True
    Config.Training.Settings.Random.cuDNN.Benchmark = True
    Config.Training.Settings.Random.Seed.Dataset.Split = 24
    Config.Training.Settings.Random.Seed.Dataset.Transform = 16
    Config.Training.Settings.Random.Seed.Dataset.Shuffle = 33
    Config.Training.Settings.Random.Seed.Model = 10
    
    Config.Logging.StepsPerLog = 8
    Config.Logging.Image.Columns = 3
    Config.Logging.Image.Rows = 10
    Config.Logging.Image.Figsize = (300,300)
    Config.Logging.Image.Fontsize = 200
    Config.Logging.Image.DPI = 10
    Config.Logging.Image.MaskAlpha = 0.6
    Config.Logging.Image.Threshold = 0.5
    
    Config.Logging.RootPath = '/kaggle/working'
    Config.Logging.FolderName = 'densenet201_non_pretrained_02'
    Config.Logging.Note = ''

In [ ]:
%run train.py

In [ ]:
import shutil
import pathlib
log_folder_path = pathlib.Path('/kaggle/working',Config.Logging.FolderName)
shutil.make_archive(log_folder_path,
                    'zip',
                    log_folder_path)